In [1]:
import json
import numpy as np
import random
import nltk
import utils as u
#nltk.download('punkt')
#nltk.download('wordnet')

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD


class ChatModel:

    def __init__(self):
        #Call tokenizing procedure
        w, words, documents, classes, self._intents = self.tokenizing('intents.json')

        #Call lemmatizing procedure
        w, words, documents, classes, lemmatizer = self.lemmatizing(w, words, documents, classes)

        #Call training_data procedure
        self._train_x, self._train_y = self.training_data(w, words, documents, classes, lemmatizer)

        #Call tokenizing procedure
        self._model = self.training(self._train_x, self._train_y)
    

    def tokenizing(self,url):
        words=[]
        classes = []
        documents = []
        intents = json.loads(open(url).read())

        for intent in intents['intents']:
            for pattern in intent['patterns']:
                #tokenize each word
                w = nltk.word_tokenize(pattern)
                words.extend(w)
                #add documents in the corpus
                documents.append((w, intent['tag']))
                # add to our classes list
                if intent['tag'] not in classes:
                    classes.append(intent['tag'])

        return w, words, documents, classes, intents

    def lemmatizing(self, w, words, documents, classes):
        ignore_words = ['?', '!']
        lemmatizer = nltk.stem.WordNetLemmatizer()

        # lemmatize, lower each word and remove duplicates
        words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]

        # sort classes and words
        classes = sorted(list(set(classes)))
        words = sorted(list(set(words)))
        # documents = combination between patterns and intents
        print (len(documents), "documents")

        # classes = intents
        print (len(classes), "classes", classes)

        # words = all words, vocabulary
        print (len(words), "unique lemmatized words", words)

        u.create_pickle(words, 'pickles\words.pkl') 
        u.create_pickle(classes, 'pickles\classes.pkl')
        return w, words, documents, classes, lemmatizer

    def training_data(self, w, words, documents, classes, lemmatizer):
        # create our training data
        training = []
        train_x = []
        train_y = []
        # create an empty array for our output
        output_empty = [0] * len(classes)

        # training set, bag of words for each sentence
        for doc in documents:
            # initialize our bag of words
            bag = []
            # list of tokenized words for the pattern
            pattern_words = doc[0]
            # lemmatize each word - create base word, in attempt to represent related words
            pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
            # create our bag of words array with 1, if word match found in current pattern

            for w in words:
                bag.append(1) if w in pattern_words else bag.append(0)

            # output is a '0' for each tag and '1' for current tag (for each pattern)
            output_row = list(output_empty)
            output_row[classes.index(doc[1])] = 1
            training.append([bag, output_row])

        # shuffle our features and turn into np.array
        random.shuffle(training)
        training = np.array(training)
        # create train and test lists. X - patterns, Y - intents
        train_x = list(training[:,0])
        train_y = list(training[:,1])

        print("Training data created")
        return train_x, train_y

    def training(self,train_x, train_y):
        #Sequential from Keras
        # Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
        # equal to number of intents to predict output intent with softmax
        model = Sequential()
        model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(len(train_y[0]), activation='softmax'))
        # Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
        sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
        #fitting and saving the model 
        hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
        model.save('chatbot_model.h5', hist)
        print("modeseql created")

        return model

    def get_train_x(self):
        return self._train_x

    def get_train_y(self):
        return self._train_y
    
    def get_model(self):
        return self._model

    def get_intents(self):
        return self._intents
        

2023-08-19 14:45:36.860403: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
class ChatApp:

    def __init__(self):
        self.cM = chatModel()
        self._lemmatizer = nltk.stem.WordNetLemmatizer()
        self._model = load_model('chatbot_model.h5')
        self._intents = self.cM.get_intents() 
        self._words = u.load_pickle('pickles\words.pkl')
        self._classes = u.load_pickle('pickles\classes.pkl')
        self._tags_=[]
        
    def clean_up_sentence(self,sentence):
        # tokenize the pattern - split words into array
        sentence_words = nltk.word_tokenize(sentence)
        # stem each word - create short form for word
        sentence_words = [self._lemmatizer.lemmatize(word.lower()) for word in sentence_words]
        return sentence_words

    # return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
    def bow(self, sentence, words, show_details=True):
        # tokenize the pattern
        sentence_words = self.clean_up_sentence(sentence)
        # bag of words - matrix of N words, vocabulary matrix
        bag = [0]*len(words) 
        for s in sentence_words:
            for i,w in enumerate(words):
                if w == s: 
                    # assign 1 if current word is in the vocabulary position
                    bag[i] = 1
                    if show_details:
                        print ("found in bag: %s" % w)
        return(np.array(bag))

    def predict_class(self, sentence, model):
        ERROR_THRESHOLD = 0.05
        # filter out predictions below a threshold
        p = self.bow(sentence, self._words, show_details=False)
        res = self._model.predict(np.array([p]))[0]
        
        results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
        # sort by strength of probability
        results.sort(key=lambda x: x[1], reverse=True)
        return_list = []
        for r in results:
            return_list.append({"intent": self._classes[r[0]], "probability": str(r[1])})
        return return_list


    def getResponse(self, ints, intents_json):
        tag = ints[0]['intent']
        context=""
        list_of_intents = intents_json['intents']
        for i in list_of_intents:
            if(i['tag']== tag and i['context']==context) or (i['tag']==tag and i['context']==""):
                result = random.choice(i['responses'])
                self._tags_.append(tag)
                break
        return result

    def chatbot_response(self, text):
        ints = self.predict_class(text, self._model)
        print(ints)
        res = self.getResponse(ints, self._intents)
        return res

In [ ]:
#Creating GUI with tkinter
import tkinter
from tkinter import *
from chatapp import ChatApp as cA

def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
            ChatLog.config(state=NORMAL)
            ChatLog.insert(END, "You: " + msg + '\n\n')
            ChatLog.config(foreground="black", font=("Arial", 16 ))
            res = cA().chatbot_response(msg)
            ChatLog.insert(END, "Fern: " + res + '\n\n')
            ChatLog.config(state=DISABLED)
            ChatLog.yview(END)


base = Tk()
base.title("FERN")
base.geometry("400x500")
base.resizable(width=TRUE, height=TRUE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="#d4e09b", font="Arial",)
ChatLog.config(state=DISABLED)
#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview)
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Arial",16,'bold'), text="Send", width="12", height=1,
                    bd=0, bg="blue", activebackground="red",fg='black',
                    command= send )
#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height=1, font="Arial")
EntryBox.bind("<Return>", send)

#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

ChatLog.config(state=NORMAL)
ChatLog.insert(END, "Fern: " + "Hello, my name is Fern and I will be assisting you today." + '\n\n'+ "I will begin with asking you a range of questions and based on the information you have provided, I will give some possible solutions. In case you would like further assistance by a lawyer, we will also provide a list of lawyers that we think are best suited for your needs." + '\n\n'+ "If you are living with the person who is inflicting harm on you, you can click the exit button at the top right to exit this website. "+'\n'
+"Does everything make sense so far?"+'\n\n')
ChatLog.config(foreground="black", font=("Arial", 16 ))

base.mainloop()

12 documents
10 classes ['consent', 'diagnosis', 'elaborate', 'family', 'frequency', 'involvement', 'length', 'physical', 'prospect', 'reciprocate']
99 unique lemmatized words [',', '.', 'a', 'about', 'abusive', 'am', 'and', 'been', 'before', 'begin', 'but', 'called', 'can', 'close', 'consent', 'continue', 'do', 'don', 'for', 'friend', 'get', 'ha', 'hand', 'happens', 'have', 'health', 'her', 'hitting', 'hurt', 'i', 'is', 'it', 'kicked', 'kid', 'know', 'make', 'marriage', 'may', 'me', 'mental', 'my', "n't", 'neighbour', 'never', 'now', 'occurring', 'off', 'on', 'once', 'only', 'our', 'partner', 'physical', 'physically', 'police', 'priority', 'safety', 'sense', 'set', 'she', 'shouting', 'so', 'something', 'sometimes', 'stab', 'start', 'started', 'stuff', 'sure', 't', 'taking', 'that', 'the', 'their', 'there', 'think', 'this', 'throwing', 'time', 'to', 'toll', 'tried', 'two', 'u', 've', 'wa', 'we', 'week', 'what', 'when', 'where', 'will', 'with', 'working', 'worried', 'year', 'yes', 'you'

/Users/adamchoong/Desktop/MAC x LSS legal hackathon/retrival based chatbot /chat_model.py:101: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)
2023-08-19 14:46:43.102678: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


3/3 [==============================] - 0s 1ms/step - loss: 2.4233 - accuracy: 0.0833
Epoch 2/200
3/3 [==============================] - 0s 872us/step - loss: 2.2963 - accuracy: 0.0833
Epoch 3/200
3/3 [==============================] - 0s 857us/step - loss: 2.2604 - accuracy: 0.1667
Epoch 4/200
3/3 [==============================] - 0s 823us/step - loss: 2.1321 - accuracy: 0.4167
Epoch 5/200
3/3 [==============================] - 0s 857us/step - loss: 1.8892 - accuracy: 0.3333
Epoch 6/200
3/3 [==============================] - 0s 807us/step - loss: 1.6982 - accuracy: 0.6667
Epoch 7/200
3/3 [==============================] - 0s 875us/step - loss: 1.9216 - accuracy: 0.4167
Epoch 8/200
3/3 [==============================] - 0s 821us/step - loss: 1.6560 - accuracy: 0.5000
Epoch 9/200
3/3 [==============================] - 0s 839us/step - loss: 1.7342 - accuracy: 0.3333
Epoch 10/200
3/3 [==============================] - 0s 945us/step - loss: 1.5914 - accuracy: 0.5833
Epoch 11/200
3/3 [=====

3/3 [==============================] - 0s 782us/step - loss: 0.0558 - accuracy: 1.0000
Epoch 84/200
3/3 [==============================] - 0s 753us/step - loss: 0.0750 - accuracy: 1.0000
Epoch 85/200
3/3 [==============================] - 0s 748us/step - loss: 0.1052 - accuracy: 1.0000
Epoch 86/200
3/3 [==============================] - 0s 844us/step - loss: 0.0515 - accuracy: 1.0000
Epoch 87/200
3/3 [==============================] - 0s 836us/step - loss: 0.2405 - accuracy: 0.9167
Epoch 88/200
3/3 [==============================] - 0s 870us/step - loss: 0.2151 - accuracy: 0.9167
Epoch 89/200
3/3 [==============================] - 0s 839us/step - loss: 0.0121 - accuracy: 1.0000
Epoch 90/200
3/3 [==============================] - 0s 933us/step - loss: 0.0629 - accuracy: 1.0000
Epoch 91/200
3/3 [==============================] - 0s 802us/step - loss: 0.0910 - accuracy: 1.0000
Epoch 92/200
3/3 [==============================] - 0s 791us/step - loss: 0.0344 - accuracy: 1.0000
Epoch 93/200


3/3 [==============================] - 0s 858us/step - loss: 0.0106 - accuracy: 1.0000
Epoch 165/200
3/3 [==============================] - 0s 812us/step - loss: 0.0800 - accuracy: 0.9167
Epoch 166/200
3/3 [==============================] - 0s 774us/step - loss: 0.2040 - accuracy: 0.9167
Epoch 167/200
3/3 [==============================] - 0s 778us/step - loss: 0.0283 - accuracy: 1.0000
Epoch 168/200
3/3 [==============================] - 0s 805us/step - loss: 0.0241 - accuracy: 1.0000
Epoch 169/200
3/3 [==============================] - 0s 897us/step - loss: 0.0927 - accuracy: 1.0000
Epoch 170/200
3/3 [==============================] - 0s 847us/step - loss: 0.0561 - accuracy: 1.0000
Epoch 171/200
3/3 [==============================] - 0s 751us/step - loss: 0.0096 - accuracy: 1.0000
Epoch 172/200
3/3 [==============================] - 0s 791us/step - loss: 0.0093 - accuracy: 1.0000
Epoch 173/200
3/3 [==============================] - 0s 817us/step - loss: 0.0467 - accuracy: 1.0000
Epoc

...

3/3 [==============================] - 0s 841us/step - loss: 1.4752 - accuracy: 0.5833
Epoch 13/200
3/3 [==============================] - 0s 825us/step - loss: 1.1399 - accuracy: 0.7500
Epoch 14/200
3/3 [==============================] - 0s 862us/step - loss: 1.3787 - accuracy: 0.6667
Epoch 15/200
3/3 [==============================] - 0s 864us/step - loss: 0.9210 - accuracy: 1.0000
Epoch 16/200
3/3 [==============================] - 0s 833us/step - loss: 1.4227 - accuracy: 0.8333
Epoch 17/200
3/3 [==============================] - 0s 797us/step - loss: 1.2152 - accuracy: 0.5833
Epoch 18/200
3/3 [==============================] - 0s 854us/step - loss: 0.8755 - accuracy: 0.9167
Epoch 19/200
3/3 [==============================] - 0s 800us/step - loss: 0.8204 - accuracy: 0.9167
Epoch 20/200
3/3 [==============================] - 0s 814us/step - loss: 0.8388 - accuracy: 1.0000
Epoch 21/200
3/3 [==============================] - 0s 896us/step - loss: 0.9996 - accuracy: 0.7500
Epoch 22/200


3/3 [==============================] - 0s 848us/step - loss: 0.0586 - accuracy: 1.0000
Epoch 95/200
3/3 [==============================] - 0s 782us/step - loss: 0.1286 - accuracy: 1.0000
Epoch 96/200
3/3 [==============================] - 0s 848us/step - loss: 0.5691 - accuracy: 0.8333
Epoch 97/200
3/3 [==============================] - 0s 827us/step - loss: 0.0309 - accuracy: 1.0000
Epoch 98/200
3/3 [==============================] - 0s 801us/step - loss: 0.2580 - accuracy: 0.9167
Epoch 99/200
3/3 [==============================] - 0s 806us/step - loss: 0.0204 - accuracy: 1.0000
Epoch 100/200
3/3 [==============================] - 0s 805us/step - loss: 0.3389 - accuracy: 0.9167
Epoch 101/200
3/3 [==============================] - 0s 749us/step - loss: 0.0793 - accuracy: 1.0000
Epoch 102/200
3/3 [==============================] - 0s 843us/step - loss: 0.0501 - accuracy: 1.0000
Epoch 103/200
3/3 [==============================] - 0s 827us/step - loss: 0.0550 - accuracy: 1.0000
Epoch 104

3/3 [==============================] - 0s 834us/step - loss: 0.0262 - accuracy: 1.0000
Epoch 176/200
3/3 [==============================] - 0s 806us/step - loss: 0.0081 - accuracy: 1.0000
Epoch 177/200
3/3 [==============================] - 0s 755us/step - loss: 0.0637 - accuracy: 1.0000
Epoch 178/200
3/3 [==============================] - 0s 812us/step - loss: 0.1557 - accuracy: 0.9167
Epoch 179/200
3/3 [==============================] - 0s 881us/step - loss: 0.0363 - accuracy: 1.0000
Epoch 180/200
3/3 [==============================] - 0s 850us/step - loss: 0.1562 - accuracy: 1.0000
Epoch 181/200
3/3 [==============================] - 0s 827us/step - loss: 0.0248 - accuracy: 1.0000
Epoch 182/200
3/3 [==============================] - 0s 843us/step - loss: 0.0616 - accuracy: 1.0000
Epoch 183/200
3/3 [==============================] - 0s 834us/step - loss: 0.0919 - accuracy: 0.9167
Epoch 184/200
3/3 [==============================] - 0s 872us/step - loss: 0.0494 - accuracy: 1.0000
Epoc

Epoch 46/200
3/3 [==============================] - 0s 762us/step - loss: 0.4046 - accuracy: 0.8333
Epoch 47/200
3/3 [==============================] - 0s 721us/step - loss: 0.1818 - accuracy: 0.9167
Epoch 48/200
3/3 [==============================] - 0s 826us/step - loss: 0.3546 - accuracy: 0.8333
Epoch 49/200
3/3 [==============================] - 0s 800us/step - loss: 0.0920 - accuracy: 1.0000
Epoch 50/200
3/3 [==============================] - 0s 772us/step - loss: 0.2258 - accuracy: 1.0000
Epoch 51/200
3/3 [==============================] - 0s 864us/step - loss: 0.1263 - accuracy: 1.0000
Epoch 52/200
3/3 [==============================] - 0s 800us/step - loss: 0.5074 - accuracy: 0.8333
Epoch 53/200
3/3 [==============================] - 0s 780us/step - loss: 0.1573 - accuracy: 1.0000
Epoch 54/200
3/3 [==============================] - 0s 793us/step - loss: 0.2510 - accuracy: 0.9167
Epoch 55/200
3/3 [==============================] - 0s 823us/step - loss: 0.2239 - accuracy: 0.9167


3/3 [==============================] - 0s 803us/step - loss: 0.0051 - accuracy: 1.0000
Epoch 128/200
3/3 [==============================] - 0s 781us/step - loss: 0.0384 - accuracy: 1.0000
Epoch 129/200
3/3 [==============================] - 0s 802us/step - loss: 0.0875 - accuracy: 1.0000
Epoch 130/200
3/3 [==============================] - 0s 835us/step - loss: 0.0705 - accuracy: 1.0000
Epoch 131/200
3/3 [==============================] - 0s 740us/step - loss: 0.0031 - accuracy: 1.0000
Epoch 132/200
3/3 [==============================] - 0s 769us/step - loss: 0.0243 - accuracy: 1.0000
Epoch 133/200
3/3 [==============================] - 0s 816us/step - loss: 0.0359 - accuracy: 1.0000
Epoch 134/200
3/3 [==============================] - 0s 696us/step - loss: 0.0044 - accuracy: 1.0000
Epoch 135/200
3/3 [==============================] - 0s 834us/step - loss: 0.0650 - accuracy: 1.0000
Epoch 136/200
3/3 [==============================] - 0s 819us/step - loss: 0.0060 - accuracy: 1.0000
Epoc